In [20]:
# !pip install ragas==0.0.22 langchain_openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 268.0 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: ragas
    Found existing installation: ragas 0.1.1
    Uninstalling ragas-0.1.1:
      Successfully uninstalled ragas-0.1.1


In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio
import os
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# IMPORTANT: Remember to create a .env variable containing: OPENAI_API_KEY=sk-xyz where xyz is your key

# Access the API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAI API client
openai.api_key = api_key

nest_asyncio.apply()

In [2]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../data/Robinson_Advisory.txt", encoding="windows-1252")
index = VectorstoreIndexCreator().from_loaders([loader])


text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=0
)
documents = loader.load_and_split()
# texts = text_splitter.split_text(document)
docs = text_splitter.split_documents(documents)

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
embeddings = OpenAIEmbeddings()

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name="contract",
    connection_args={"host": "127.0.0.1", "port": "19530", "database": "contract"},
)
vector_db = Milvus(
    embeddings,
    connection_args={"host": "127.0.0.1", "port": "19530"},
    collection_name="contract",
)


In [4]:
llm = ChatOpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever(), 
    return_source_documents=True,
)

In [5]:
# testing it out

question = "Who are the parties to the Agreement and what are their defined names?"
result = qa_chain({"query": question})
result["result"]

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The parties to the Agreement are the Sellers and the Buyer. Their defined names are not explicitly mentioned in the provided context.'

In [6]:
eval_questions = [
    "Who are the parties to the Agreement and what are their defined names?",
    "What is the termination notice?",
    " What are the payments to the Advisor under the Agreement?",
    "Can the Agreement or any of its obligations be assigned?",
    "Who owns the IP?",
    "Is there a non-compete obligation to the Advisor?",
    "Can the Advisor charge for meal time?",
    "In which street does the Advisor live?",
    "Is the Advisor entitled to social benefits?",
    "What happens if the Advisor claims compensation based on employment relationship with the Company?",
]

eval_answers = [
    "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)",
    "According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement.",
    " According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance.",
    "1. Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9 the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking.",
    "According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company.",
    "Yes. During the term of engagement with the Company and for a period of 12 months thereafter.",
    "No. See Section 6.1, Billable Hour doesn’t include meals or travel time.",
    "1 Rabin st, Tel Aviv, Israel",
    "No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company.",
    " If the Advisor is determined to be an employee of the Company by a governmental authority, payments to the Advisor will be retroactively reduced so that 60% constitutes salary payments and 40% constitutes payment for statutory rights and benefits. The Company may offset any amounts due to the Advisor from any amounts payable under the Agreement. The Advisor must indemnify the Company for any losses or expenses incurred if an employer/employee relationship is determined to exist.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [7]:
result = qa_chain({"query": eval_questions[1]})
result["result"]

"The termination notice required in this agreement is fourteen (14) days' prior written notice."

In [8]:
result = qa_chain(examples[4])
result["result"]

'Based on the provided context, it is stated that all Intellectual Property and Technology were developed at private expense, and no Institutions have obtained rights that would affect the commercial value of the IP. Therefore, the Acquired Company or its employees, former employees, contractors, or consultants who developed the Intellectual Property own the IP.'

In [9]:
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# create evaluation chains
faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
context_rel_chain = RagasEvaluatorChain(metric=context_precision)
context_recall_chain = RagasEvaluatorChain(metric=context_recall)

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Recheck the result that we are going to validate.
result

{'query': 'Who owns the IP?',
 'ground_truths': ['According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company.'],
 'result': "The IP, which includes any Work Product created, is fully and exclusively owned by the Company according to the terms of the Undertaking. The Advisor is required to sign any document or perform any action needed to formalize the Company's ownership of the IP.",
 'source_documents': [Document(page_content='-  4- \n \nConfidentiality, None Compete and IP Ownership Undertaking  \nAppendix A to Advisory Service Agreement  as of June  15th, 2023  \n \nTHIS CONFIDENTIALITY UNDERTAKING (“ Undertaking ”) is entered into as of June  15th, 2023 (“ Effective Date ”), by Mr. Jack Robinson,  \nPassport Number 780055578 , residing at 1 Rabin st , Tel Aviv, Israel, Email: jackrobinson@gmail.com , (“Advisor ”), towards Cloud  Investments \nLtd (“ Company ”), as follows:  \n1. Defin itions:  (a) Com

In [11]:
eval_result = faithfulness_chain(result)
eval_result["faithfulness_score"]

1.0

In [12]:
fake_result = result.copy()
fake_result["result"] = "we are the champions"
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness_score"]

0.0

In [13]:
eval_result = context_recall_chain(result)
eval_result["context_recall_score"]

1.0

In [14]:
from langchain.schema import Document

fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall_score"]

1.0

In [12]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

evaluating...


100%|██████████| 1/1 [00:18<00:00, 18.88s/it]


[{'faithfulness_score': 0.6666666666666666},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0}]

In [13]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

print("evaluating...")
x = context_recall_chain.evaluate(examples, predictions)
x

evaluating...


0it [00:00, ?it/s]


[]

In [14]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r

evaluating...


0it [00:00, ?it/s]


[]